<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin-bottom: 20px;">
    <h1 style="color: white; margin: 0; font-size: 36px;">📝 Notebook 2: Signatures</h1>
    <p style="color: rgba(255,255,255,0.9); margin-top: 10px; font-size: 18px;">Defining What You Want from Language Models</p>
</div>

<div style="display: flex; justify-content: space-between; margin-bottom: 20px;">
    <a href="01_hello_logillm.ipynb" style="text-decoration: none; padding: 10px 20px; background: #f0f0f0; border-radius: 5px;">← Notebook 1</a>
    <span style="padding: 10px 20px; background: #e8f5e9; border-radius: 5px;">🟢 Beginner • 15 minutes</span>
    <a href="03_modules.ipynb" style="text-decoration: none; padding: 10px 20px; background: #f0f0f0; border-radius: 5px;">Notebook 3 →</a>
</div>

## 🎯 What You'll Learn

<div style="background: #f5f5f5; padding: 20px; border-radius: 10px; border-left: 4px solid #667eea;">
    <ul style="margin: 0; padding-left: 20px;">
        <li>✅ <strong>Master string signatures</strong> for quick prototyping</li>
        <li>✅ <strong>Build class-based signatures</strong> with rich types and validation</li>
        <li>✅ <strong>Use field descriptors</strong> to guide LLM behavior</li>
        <li>✅ <strong>Work with complex types</strong> like lists, dicts, and custom objects</li>
        <li>✅ <strong>Apply constraints and validation</strong> to ensure quality outputs</li>
        <li>✅ <strong>Create reusable signature libraries</strong> for your domain</li>
    </ul>
</div>

## 🔧 Setup

In [1]:
import asyncio
import json
from typing import List, Dict, Optional
from enum import Enum

from logillm.core.predict import Predict
from logillm.core.signatures import Signature, InputField, OutputField
from logillm.providers import create_provider, register_provider

# Setup provider
provider = create_provider("openai", model="gpt-4o-mini")
register_provider(provider, set_default=True)

print("✅ LogiLLM ready!")

✅ LogiLLM ready!


## 📚 Part 1: String Signatures - Quick & Flexible

String signatures are the fastest way to define transformations. They follow a simple pattern:

<div style="background: #e3f2fd; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <code style="font-size: 18px;">"input1, input2, ... -> output1, output2, ..."</code>
</div>

### Basic Transformations

In [2]:
# Single input -> Single output
translator = Predict("english_text -> spanish_translation")
result = await translator(english_text="Hello, how are you?")
print(f"🇪🇸 Spanish: {result.outputs['spanish_translation']}")

# Multiple inputs -> Single output
calculator = Predict("num1, num2, operation -> result")
result = await calculator(num1=15, num2=7, operation="multiply")
print(f"\n🔢 Result: {result.outputs['result']}")

# Single input -> Multiple outputs
analyzer = Predict("text -> language, word_count, sentiment")
result = await analyzer(text="Bonjour! Cette bibliothèque est fantastique!")
print(f"\n📊 Analysis:")
for key, value in result.outputs.items():
    print(f"  {key}: {value}")

🇪🇸 Spanish: Hola, ¿cómo estás?

🔢 Result: 105

📊 Analysis:
  language: French
  word_count: 6
  sentiment: Positive


### Adding Type Hints to String Signatures

In [3]:
# Type hints help the LLM understand expected formats
typed_analyzer = Predict(
    "product_description: str -> "
    "price: float, "
    "in_stock: bool, "
    "categories: list[str]"
)

result = await typed_analyzer(
    product_description="Premium wireless headphones with noise cancellation"
)

print("🛍️ Product Analysis:")
print(f"  Price: ${result.outputs['price']}")
print(f"  In Stock: {'✅' if result.outputs['in_stock'] else '❌'}")
print(f"  Categories: {', '.join(result.outputs['categories'])}")
print(f"\n  Types: {[(k, type(v).__name__) for k, v in result.outputs.items()]}")

🛍️ Product Analysis:
  Price: $299.99
  In Stock: ✅
  Categories: electronics, audio, headphones

  Types: [('price', 'float'), ('in_stock', 'bool'), ('categories', 'list')]


## 🏗️ Part 2: Class-Based Signatures - Structured & Powerful

Class-based signatures give you more control and better documentation:

### Basic Class Signature

In [4]:
class NewsArticle(Signature):
    """Analyze news articles for key information."""
    
    # Input fields
    headline: str = InputField(desc="The article headline")
    content: str = InputField(desc="The article body text")
    
    # Output fields with descriptions
    summary: str = OutputField(desc="2-3 sentence summary")
    category: str = OutputField(desc="News category: politics, technology, sports, etc.")
    sentiment: str = OutputField(desc="Overall tone: positive, negative, or neutral")
    key_entities: list[str] = OutputField(desc="Important people, places, or organizations mentioned")

# Use the signature
news_analyzer = Predict(NewsArticle)

result = await news_analyzer(
    headline="Tech Giant Announces Revolutionary AI Breakthrough",
    content="""Silicon Valley's leading tech company unveiled a new artificial intelligence 
    system today that promises to transform healthcare. The CEO demonstrated how the 
    system can diagnose diseases with 99% accuracy. Investors responded positively, 
    with stock prices rising 15%."""
)

print("📰 Article Analysis:")
print(f"\n📝 Summary: {result.outputs['summary']}")
print(f"\n🏷️ Category: {result.outputs['category']}")
print(f"😊 Sentiment: {result.outputs['sentiment']}")
print(f"\n👥 Key Entities:")
for entity in result.outputs['key_entities']:
    print(f"  • {entity}")

📰 Article Analysis:

📝 Summary: A leading tech company in Silicon Valley has introduced a groundbreaking AI system that claims to diagnose diseases with 99% accuracy, aiming to revolutionize healthcare. Following the announcement, the company's stock prices increased by 15%, indicating a positive response from investors.

🏷️ Category: Technology / Healthcare
😊 Sentiment: Positive

👥 Key Entities:
  • Silicon Valley
  • tech company
  • artificial intelligence system
  • CEO
  • healthcare
  • investors
  • stock prices


### Using Enums for Constrained Choices

In [5]:
from enum import Enum

class Priority(str, Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

class TaskStatus(str, Enum):
    TODO = "todo"
    IN_PROGRESS = "in_progress"
    BLOCKED = "blocked"
    DONE = "done"

class TaskAnalysis(Signature):
    """Analyze task descriptions and assign metadata."""
    
    task_description: str = InputField(desc="Description of the task")
    
    # Using enums ensures valid values
    priority: Priority = OutputField(desc="Task priority level")
    estimated_hours: float = OutputField(desc="Estimated hours to complete")
    suggested_status: TaskStatus = OutputField(desc="Recommended initial status")
    dependencies: list[str] = OutputField(desc="What this task depends on")
    skills_required: list[str] = OutputField(desc="Skills needed to complete this task")

task_analyzer = Predict(TaskAnalysis)

result = await task_analyzer(
    task_description="""Fix the critical bug in the payment processing system that's 
    causing transactions to fail for premium customers. Requires database access 
    and knowledge of the Stripe API."""
)

print("📋 Task Analysis:")
print(f"  Priority: {result.outputs['priority']} {'🔥' if result.outputs['priority'] == 'critical' else ''}")
print(f"  Estimated Hours: {result.outputs['estimated_hours']}h")
print(f"  Status: {result.outputs['suggested_status']}")
print(f"  Dependencies: {', '.join(result.outputs['dependencies']) if result.outputs['dependencies'] else 'None'}")
print(f"  Skills Required: {', '.join(result.outputs['skills_required'])}")

📋 Task Analysis:
  Priority: High 
  Estimated Hours: 4.0h
  Status: In Progress
  Dependencies: Access to the database, Understanding of the Stripe API
  Skills Required: Database management, Stripe API knowledge, Problem-solving skills


## 🎨 Part 3: Complex Types & Nested Structures

LogiLLM handles complex data structures elegantly:

In [6]:
from typing import Dict, List, Optional
from datetime import datetime

class Meeting(Signature):
    """Extract structured meeting information from notes."""
    
    meeting_notes: str = InputField(desc="Raw meeting notes or transcript")
    
    # Complex nested output structure
    meeting_title: str = OutputField(desc="Title or main topic of the meeting")
    attendees: List[str] = OutputField(desc="List of attendee names")
    
    # Dictionary for action items - be explicit about structure
    action_items: List[Dict[str, str]] = OutputField(
        desc="List of dictionaries, each with keys 'task' (what to do), 'owner' (who does it), and 'deadline' (when due)"
    )
    
    # Nested structure for decisions
    decisions: List[Dict[str, str]] = OutputField(
        desc="List of dictionaries, each with keys 'decision' (what was decided) and 'rationale' (why)"
    )
    
    next_steps: List[str] = OutputField(desc="Next steps agreed upon")
    follow_up_date: Optional[str] = OutputField(desc="Date for follow-up meeting if mentioned")

meeting_analyzer = Predict(Meeting)

result = await meeting_analyzer(
    meeting_notes="""
    Product Planning Meeting - March 15, 2024
    
    Attendees: Sarah (PM), John (Eng Lead), Maria (Design), Alex (QA)
    
    Discussion:
    - Sarah presented the Q2 roadmap
    - John raised concerns about the timeline for the AI feature
    - Maria showed mockups for the new dashboard
    
    Decisions:
    - We'll launch the dashboard redesign in April (users have been requesting this)
    - AI feature pushed to Q3 (need more research time)
    
    Action Items:
    - John to create technical spec for dashboard by March 20
    - Maria to finalize designs by March 18
    - Alex to prepare test plan by March 22
    - Sarah to update stakeholders on timeline changes by EOD today
    
    Next meeting scheduled for March 22 to review progress.
    """
)

print("📅 Meeting Summary:")
print(f"\n**{result.outputs['meeting_title']}**")
print(f"\n👥 Attendees: {', '.join(result.outputs['attendees'])}")

print("\n✅ Action Items:")
for item in result.outputs['action_items']:
    # Handle both dict and string cases for robustness
    if isinstance(item, dict):
        print(f"  • {item.get('task', 'N/A')}")
        print(f"    Owner: {item.get('owner', 'TBD')} | Due: {item.get('deadline', 'ASAP')}")
    else:
        # If it's a string, just print it
        print(f"  • {item}")

print("\n🎯 Decisions:")
for decision in result.outputs['decisions']:
    if isinstance(decision, dict):
        print(f"  • {decision.get('decision', '')}")
        print(f"    Rationale: {decision.get('rationale', '')}")
    else:
        print(f"  • {decision}")

if result.outputs.get('follow_up_date'):
    print(f"\n📆 Follow-up: {result.outputs['follow_up_date']}")

📅 Meeting Summary:

**Product Planning Meeting - March 15, 2024**

👥 Attendees: Sarah (PM), John (Eng Lead), Maria (Design), Alex (QA)

✅ Action Items:
  • John to create technical spec for dashboard by March 20
  • Maria to finalize designs by March 18
  • Alex to prepare test plan by March 22
  • Sarah to update stakeholders on timeline changes by EOD today

🎯 Decisions:
  • We'll launch the dashboard redesign in April
  • AI feature pushed to Q3

📆 Follow-up: March 22, 2024


## 🔍 Part 4: Field Descriptors - Guiding LLM Behavior

Field descriptions are crucial for getting exactly what you want:

In [7]:
class PreciseAnalysis(Signature):
    """Demonstrate how field descriptions guide LLM behavior."""
    
    text: str = InputField(desc="Text to analyze")
    
    # Vague description vs precise description
    vague_summary: str = OutputField(desc="Summary")  # Vague
    
    precise_summary: str = OutputField(
        desc="Exactly 2 sentences, 30-40 words total, focusing on the main point and key supporting detail"
    )  # Precise!
    
    # Controlling format with descriptions
    keywords: list[str] = OutputField(
        desc="Exactly 5 keywords, lowercase, single words only, ordered by importance"
    )
    
    # Controlling content with descriptions
    technical_level: int = OutputField(
        desc="Technical complexity from 1 (elementary) to 10 (PhD level)"
    )
    
    target_audience: str = OutputField(
        desc="Single most appropriate audience: general, technical, business, academic, or children"
    )

analyzer = Predict(PreciseAnalysis)

test_text = """Quantum computers use quantum bits or 'qubits' instead of regular bits. 
While regular bits are either 0 or 1, qubits can be in a 'superposition' of both states 
simultaneously. This allows quantum computers to process many calculations at once, 
potentially solving certain problems much faster than classical computers."""

result = await analyzer(text=test_text)

print("🔬 Analysis Comparison:")
print("\n📝 Vague Summary:")
print(f"  {result.outputs['vague_summary']}")
print(f"  (Length: {len(result.outputs['vague_summary'].split())} words)")

print("\n✨ Precise Summary:")
print(f"  {result.outputs['precise_summary']}")
print(f"  (Length: {len(result.outputs['precise_summary'].split())} words)")

print(f"\n🏷️ Keywords: {result.outputs['keywords']}")
print(f"📊 Technical Level: {result.outputs['technical_level']}/10")
print(f"👥 Target Audience: {result.outputs['target_audience']}")

🔬 Analysis Comparison:

📝 Vague Summary:
  Quantum computers operate using qubits, allowing them to perform multiple calculations simultaneously.
  (Length: 12 words)

✨ Precise Summary:
  Quantum computers utilize quantum bits, or qubits, which can exist in a superposition of states (0 and 1) at the same time, enabling them to handle many calculations concurrently and solve specific problems faster than classical computers.
  (Length: 37 words)

🏷️ Keywords: ['Quantum computers', 'qubits', 'superposition', 'calculations', 'classical computers']
📊 Technical Level: 4/10
👥 Target Audience: Individuals with a basic understanding of computer science and quantum mechanics


## 🛡️ Part 5: Validation & Constraints

You can add validation to ensure outputs meet your requirements:

In [8]:
from typing import Optional
import re

class ValidatedCustomer(Signature):
    """Extract customer information with validation constraints."""
    
    raw_text: str = InputField(desc="Raw customer information")
    
    # Fields with format constraints in descriptions
    name: str = OutputField(desc="Full name (2-4 words, letters only)")
    
    email: str = OutputField(
        desc="Valid email address in format: user@domain.com"
    )
    
    phone: str = OutputField(
        desc="US phone number in format: (XXX) XXX-XXXX"
    )
    
    age: int = OutputField(
        desc="Age in years (must be between 18 and 120)"
    )
    
    account_type: str = OutputField(
        desc="Account type: exactly one of 'basic', 'premium', or 'enterprise'"
    )
    
    credit_score: Optional[int] = OutputField(
        desc="Credit score between 300-850 if mentioned, otherwise None"
    )

customer_extractor = Predict(ValidatedCustomer)

result = await customer_extractor(
    raw_text="""New customer John Michael Smith, 34 years old, reached out via 
    jsmith@techcorp.com. His phone is 555-123-4567 (area code 415). He's interested 
    in our premium plan. Mentioned his credit score is around 750."""
)

# Validate the outputs
print("✅ Extracted Customer Data:")
outputs = result.outputs

# Check name format
name_valid = len(outputs['name'].split()) in [2, 3, 4]
print(f"  Name: {outputs['name']} {'✓' if name_valid else '✗'}")

# Check email format
email_valid = '@' in outputs['email'] and '.' in outputs['email']
print(f"  Email: {outputs['email']} {'✓' if email_valid else '✗'}")

# Check phone format
phone_pattern = r'\(\d{3}\) \d{3}-\d{4}'
phone_valid = bool(re.match(phone_pattern, outputs['phone']))
print(f"  Phone: {outputs['phone']} {'✓' if phone_valid else '✗'}")

# Check age range
age_valid = 18 <= outputs['age'] <= 120
print(f"  Age: {outputs['age']} {'✓' if age_valid else '✗'}")

# Check account type
account_valid = outputs['account_type'] in ['basic', 'premium', 'enterprise']
print(f"  Account Type: {outputs['account_type']} {'✓' if account_valid else '✗'}")

# Check credit score
if outputs['credit_score']:
    credit_valid = 300 <= outputs['credit_score'] <= 850
    print(f"  Credit Score: {outputs['credit_score']} {'✓' if credit_valid else '✗'}")

all_valid = all([name_valid, email_valid, phone_valid, age_valid, account_valid])
print(f"\n{'✅ All validations passed!' if all_valid else '⚠️ Some validations failed'}")

✅ Extracted Customer Data:
  Name: John Michael Smith ✓
  Email: jsmith@techcorp.com ✓
  Phone: 555-123-4567 ✗
  Age: 34 ✓
  Account Type: premium ✓
  Credit Score: 750 ✓

⚠️ Some validations failed


## 📚 Part 6: Building a Signature Library

Create reusable signatures for your domain:

In [9]:
# Create a library of common signatures for a specific domain
class EcommerceSigatures:
    """Reusable signatures for e-commerce applications."""
    
    class ProductDescription(Signature):
        """Generate compelling product descriptions."""
        product_name: str = InputField()
        features: list[str] = InputField()
        target_audience: str = InputField()
        
        title: str = OutputField(desc="Catchy product title (max 60 chars)")
        description: str = OutputField(desc="Engaging description (100-150 words)")
        bullet_points: list[str] = OutputField(desc="5 key selling points")
        seo_keywords: list[str] = OutputField(desc="10 SEO keywords")
    
    class ReviewAnalysis(Signature):
        """Analyze customer reviews."""
        review_text: str = InputField()
        
        rating: int = OutputField(desc="Predicted rating 1-5 stars")
        sentiment: str = OutputField(desc="positive, negative, or mixed")
        pros: list[str] = OutputField(desc="Positive aspects mentioned")
        cons: list[str] = OutputField(desc="Negative aspects mentioned")
        verified_purchase: bool = OutputField(desc="Likely a real purchase?")
    
    class OrderIntent(Signature):
        """Understand customer order intentions."""
        customer_message: str = InputField()
        
        intent: str = OutputField(desc="order, cancel, modify, track, or complain")
        urgency: str = OutputField(desc="high, medium, or low")
        order_id: Optional[str] = OutputField(desc="Order ID if mentioned")
        requested_action: str = OutputField(desc="Specific action requested")

# Use the library
product_writer = Predict(EcommerceSigatures.ProductDescription)

result = await product_writer(
    product_name="EcoSmart Water Bottle",
    features=["Keeps drinks cold for 24 hours", "BPA-free", "Leak-proof", "1 liter capacity"],
    target_audience="Environmentally conscious athletes"
)

print("🛍️ Generated Product Content:")
print(f"\n📌 Title: {result.outputs['title']}")
print(f"\n📝 Description:\n{result.outputs['description']}")
print(f"\n✨ Key Features:")
for point in result.outputs['bullet_points']:
    print(f"  • {point}")
print(f"\n🔍 SEO Keywords: {', '.join(result.outputs['seo_keywords'][:5])}...")

🛍️ Generated Product Content:

📌 Title: EcoSmart Water Bottle - Stay Hydrated, Stay Green!

📝 Description:
Quench your thirst and your eco-conscious spirit with the EcoSmart Water Bottle. Engineered for the environmentally conscious athlete, this sleek and stylish water bottle is perfect for those who want to stay refreshed without harming the planet. With its impressive ability to keep drinks cold for up to 24 hours and a generous 1-liter capacity, you can tackle your toughest workouts without worrying about hydration. Made from BPA-free materials and designed with a leak-proof cap, the EcoSmart Water Bottle makes it easy to hydrate responsibly, wherever your adventures take you.

✨ Key Features:
  • Keeps drinks cold for 24 hours - perfect for long workouts or outdoor adventures
  • Made from BPA-free materials for a healthy, eco-friendly choice
  • Leak-proof design ensures no spills or mess in your gym bag
  • Generous 1-liter capacity to keep you hydrated during intense activities

## 🎮 Interactive Exercise: Design Your Signature

Create a signature for your own use case:

In [10]:
# Exercise: Create a signature for a resume analyzer
# TODO: Complete this signature

class ResumeAnalyzer(Signature):
    """Analyze resumes for key information and job fit."""
    
    # Input fields
    resume_text: str = InputField(desc="Full resume text")
    job_description: str = InputField(desc="Target job description")
    
    # TODO: Add these output fields:
    # 1. years_experience: int - Total years of experience
    # 2. top_skills: list[str] - Top 5 relevant skills
    # 3. education_level: str - Highest education (BS, MS, PhD, etc.)
    # 4. job_fit_score: float - Match score 0-1
    # 5. missing_skills: list[str] - Required skills not found
    # 6. recommendation: str - hire, maybe, or pass
    
    # Your solution here:
    years_experience: int = OutputField(desc="Total years of professional experience")
    top_skills: list[str] = OutputField(desc="Top 5 most relevant skills for the job")
    education_level: str = OutputField(desc="Highest education: HS, BS, MS, PhD, or other")
    job_fit_score: float = OutputField(desc="Job fit score from 0.0 to 1.0")
    missing_skills: list[str] = OutputField(desc="Required skills not found in resume")
    recommendation: str = OutputField(desc="Recommendation: hire, maybe, or pass")

# Test your analyzer
resume_analyzer = Predict(ResumeAnalyzer)

result = await resume_analyzer(
    resume_text="""John Doe - Software Engineer
    5 years Python development, 3 years team lead
    MS Computer Science, Stanford
    Skills: Python, React, Docker, AWS, Machine Learning
    Previous: Senior Developer at TechCorp (3 years)""",
    
    job_description="""Senior ML Engineer needed.
    Requirements: Python, TensorFlow, MLOps, 5+ years experience
    Nice to have: PhD, published papers"""
)

print("📄 Resume Analysis:")
print(f"  Experience: {result.outputs['years_experience']} years")
print(f"  Education: {result.outputs['education_level']}")
print(f"  Fit Score: {result.outputs['job_fit_score']:.0%}")
print(f"  Top Skills: {', '.join(result.outputs['top_skills'][:3])}")
print(f"  Missing: {', '.join(result.outputs['missing_skills']) if result.outputs['missing_skills'] else 'None'}")
print(f"  Recommendation: {result.outputs['recommendation'].upper()}")

📄 Resume Analysis:
  Experience: 5 years
  Education: MS
  Fit Score: 6000%
  Top Skills: Python, React, Docker
  Missing: TensorFlow, MLOps, PhD, published papers
  Recommendation: CONSIDER FOR A ROLE FOCUSED ON SOFTWARE DEVELOPMENT OR ENTRY-LEVEL ML ROLES; WILL NEED ADDITIONAL TRAINING IN TENSORFLOW AND MLOPS FOR THE SENIOR ML ENGINEER POSITION.


## 🔬 Advanced: Dynamic Signature Creation

Create signatures programmatically based on runtime conditions:

In [11]:
def create_dynamic_signature(fields: list[str], output_type: str = "analysis"):
    """Create a signature dynamically based on requirements."""
    
    # Build the signature string
    inputs = ", ".join(fields)
    
    if output_type == "analysis":
        outputs = "summary: str, key_points: list[str], score: float"
    elif output_type == "classification":
        outputs = "category: str, confidence: float, reasoning: str"
    else:  # extraction
        outputs = "entities: list[str], relationships: list[str], metadata: dict"
    
    signature_str = f"{inputs} -> {outputs}"
    return signature_str

# Example: Create different signatures based on data type
data_types = {
    "financial": ["revenue", "expenses", "profit_margin"],
    "medical": ["symptoms", "patient_history", "test_results"],
    "legal": ["case_details", "precedents", "jurisdiction"]
}

# Dynamically create and use a signature
domain = "financial"
sig = create_dynamic_signature(data_types[domain], "analysis")
print(f"📊 Dynamic Signature for {domain}:")
print(f"  {sig}")

# Use the dynamic signature
analyzer = Predict(sig)
result = await analyzer(
    revenue="$1.2M",
    expenses="$800K",
    profit_margin="33%"
)

print(f"\n📈 Analysis Results:")
print(f"  Summary: {result.outputs['summary']}")
print(f"  Score: {result.outputs['score']}")

📊 Dynamic Signature for financial:
  revenue, expenses, profit_margin -> summary: str, key_points: list[str], score: float

📈 Analysis Results:
  Summary: The business generated a revenue of $1.2 million, with expenses amounting to $800,000, resulting in a profit margin of 33%. This indicates a profitable operation.
  Score: 8.5


## 📊 Signature Best Practices

<div style="background: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h3 style="margin-top: 0;">✅ DO:</h3>
    <ul>
        <li><strong>Be specific</strong> in field descriptions</li>
        <li><strong>Use type hints</strong> for better parsing</li>
        <li><strong>Provide examples</strong> in descriptions when format matters</li>
        <li><strong>Use enums</strong> for constrained choices</li>
        <li><strong>Keep signatures focused</strong> on a single task</li>
    </ul>    
    <h3>❌ DON'T:</h3>
    <ul>
        <li><strong>Make fields too vague</strong> (e.g., "output: str")</li>
        <li><strong>Mix unrelated tasks</strong> in one signature</li>
        <li><strong>Forget validation</strong> for critical fields</li>
        <li><strong>Use overly complex nested structures</strong> unnecessarily</li>
    </ul>
</div>

## 🎯 Summary & Key Takeaways

<div style="background: #e8f5e9; padding: 25px; border-radius: 10px; margin: 20px 0;">
    <h3 style="margin-top: 0;">What You've Learned</h3>
    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
        <div>
            <h4>✅ Signature Types</h4>
            <ul>
                <li>String signatures for quick prototypes</li>
                <li>Class signatures for production code</li>
                <li>Dynamic signatures for flexibility</li>
                <li>Signature libraries for reusability</li>
            </ul>
        </div>
        <div>
            <h4>✅ Advanced Features</h4>
            <ul>
                <li>Type hints and validation</li>
                <li>Field descriptions as prompts</li>
                <li>Complex nested structures</li>
                <li>Enums and constraints</li>
            </ul>
        </div>
    </div>
</div>

## 🏁 Progress Check

In [12]:
# Progress tracker
completed = {
    "string_signatures": True,
    "class_signatures": True,
    "complex_types": True,
    "field_descriptors": True,
    "validation": True,
    "signature_library": True,
    "exercise": True,
    "dynamic_signatures": True
}

total = len(completed)
done = sum(completed.values())
percentage = (done / total) * 100

print(f"📊 Notebook 2 Progress: {done}/{total} sections ({percentage:.0f}%)")
print("\n" + "█" * int(percentage // 5) + "░" * (20 - int(percentage // 5)))

if percentage == 100:
    print("\n🎉 Excellent! You've mastered LogiLLM Signatures!")
    print("Ready for Notebook 3: Modules - Making Things Happen")

📊 Notebook 2 Progress: 8/8 sections (100%)

████████████████████

🎉 Excellent! You've mastered LogiLLM Signatures!
Ready for Notebook 3: Modules - Making Things Happen


<div style="display: flex; justify-content: space-between; margin-top: 40px; padding: 20px; background: #f5f5f5; border-radius: 10px;">
    <a href="01_hello_logillm.ipynb" style="text-decoration: none; padding: 10px 20px; background: white; border-radius: 5px; border: 1px solid #ddd;">← Notebook 1</a>
    <div style="text-align: center;">
        <strong>Excellent work! You're now a Signature expert! 🎓</strong>
    </div>
    <a href="03_modules.ipynb" style="text-decoration: none; padding: 10px 20px; background: #667eea; color: white; border-radius: 5px;">Continue to Notebook 3 →</a>
</div>